# Sudoku solver with statistics

Author: Frankie Inguanez <br />
Date: 15/01/2023<br /><br />

An backtracking algorithm for a 9x9 sudoku puzzle taking.

In [93]:
import sudokuSearchAlg as ssa
import sudokuGuessAlg as sga
import sudokuPuzzleUtils as spu

In [94]:
class SudokuExecutor:
    def __init__(self, puzzlesFileName: str, statsFileName: str, errorsFileName: str, limit: int):
        self.puzzlesFileName=puzzlesFileName
        self.statsFileName=statsFileName
        self.errorsFileName=errorsFileName
        self.limit=limit

In [95]:
class SudokuConfig:
    def __init__(self, searchMode: int, guessMode: int):
        self.searchMode=searchMode
        self.guessMode=guessMode

In [96]:
class SudokuStats:
    def __init__(self):
        self.guesses = 0
        self.backtracks = 0
        self.executionTime = None
        self.unknowns = None

    def incrementGuesses(self):
        self.guesses += 1

    def incrementBacktracks(self):
        self.backtracks += 1

    def registerExecutionTime(self, executionTime):
        self.executionTime=executionTime

    def setUnknowns(self, zeros:int):
        self.unknowns=zeros

In [97]:
def backtracking(board, stats: SudokuStats, config: SudokuConfig):
    """
    Solves a 9x9 sudoku puzzle using backtracking algorithm.
    Arguments:
        board: the 9x9 puzzle to be solved.
        stats: The statistics object to record algorithm.
        searchMode: defines how the puzzle is parsed: 1 by row; 2 by col; 3 by box sequentially; 4 by box in a zig-zag; 5 by box in a spiral; 6 by box in a semi-zig-zag
        guessMode: defines how numbers are guessed: 1 sequentially; 2 randomly
    """
    # Find the next empty cell
    find = ssa.findEmpty(board, config.searchMode)

    # If there is no empty cell than puzzle is complete
    if not find:
        return True
    else:
        row, col = find

    # Get numbers to guess and attempt
    for guess in sga.getGuesses(board, config.guessMode):
        if spu.isValid(board, guess, (row, col)):

            # Brute force guess
            stats.incrementGuesses()
            board[row][col] = guess

            # Attempt to solve rest of puzzle with current choice
            if backtracking(board, stats, config):
                return True

            # Invalid puzzle so backtrack
            stats.incrementBacktracks()
            board[row][col] = 0

    return False

In [113]:
board = spu.to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')
board

[[1, 0, 0, 0, 0, 8, 5, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 6, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 5, 0, 2, 0, 0, 4, 7],
 [0, 9, 4, 6, 0, 0, 3, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 9, 0],
 [9, 2, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 5]]

In [114]:
# Register the start time
config = SudokuConfig(searchMode=6, guessMode=1)

print("Solved puzzle!" if backtracking(board, SudokuStats(), config) else "Could not solve puzzle")

Solved puzzle!


In [115]:
import timeit

stats = SudokuStats()

timeit.timeit(lambda: backtracking(board, stats, config), number=100000)

0.9082337000290863

In [116]:
stats = SudokuStats()
stats.registerExecutionTime(timeit.timeit(lambda: backtracking(board, stats, config), number=100000))

In [119]:
def solvePuzzles(executor: SudokuExecutor, config: SudokuConfig):
    """
    Solves puzzles found in a file using backtracking algorithm.
    Arguments:
        puzzlesFileName: file containing puzzles in following format: <id>, <puzzle>, <solution>
        statsFileName: file where statistics shall be saved.
        errorsFileName: file where errors shall be saved.
        limit: the limit number of puzzles to solve.
        searchMode: defines how missing values are searched.
        guessMode: defines how guesses are made.
    """
    import tqdm
    import timeit

    if not executor.limit:
        limit = spu.getFileLineCount(executor.puzzlesFileName)
    else: limit = executor.limit

    i = 1
    hasError = False
    try:
        print("Starting sudoku base solver.")
        
        # Open statistics file
        with open(executor.statsFileName, "w", encoding="utf-8") as sf:
            #Write header row
            sf.write("Puzzle,Solution,Execution Time,Zeros,Guesses,Backtracks\n")

            # Open puzzles and read till limit is reached.
            with open(executor.puzzlesFileName, "r", encoding="utf-8") as pf:
                for line in tqdm.tqdm(pf, total=limit):
                    # Stop at limit
                    if (i>limit):
                        break
                    
                    # Parse the content
                    data = line.split(',')
                    p = data[1].strip()

                    # Create board and statistics
                    board = spu.to2DArray(p)
                    
                    stats = SudokuStats();
                    stats.setUnknowns(p.count('0'))
                    stats.registerExecutionTime(timeit.timeit(lambda: backtracking(board, stats, config), number=100000))                   
                        
                    # Write statistics
                    sf.write("{},{},{:0.17f},{:0.0f},{:0.0f},{:0.0f}\n".format(p, spu.toStr(board), stats.executionTime, stats.unknowns, stats.guesses, stats.backtracks))
                    i+=1

    except Exception as e:
        hasError = True
        spu.saveError(e, executor.errorsFileName)

    print("Operation encountered some errors. Check {} for details or script output above.".format(executor.errorsFileName) \
        if hasError else "Sudoku puzzles solved completed successfully.")
    print("Sudoku solver statistics saved in {}".format(executor.statsFileName))

In [120]:
config = SudokuConfig(searchMode=1, guessMode=1)
exec = SudokuExecutor(puzzlesFileName="sudokuDataset.csv",statsFileName="backtracking.csv", errorsFileName="backtrackingErrors.txt", limit=10)

solvePuzzles(exec, config)

Starting sudoku base solver.


100%|██████████| 10/10 [00:04<00:00,  2.27it/s]

Sudoku puzzles solved completed successfully.
Sudoku solver statistics saved in backtracking.csv
